# iTitle

**Цель**: сделать веб-приложение, помогающее авторам и редакторам IT-публикаций подбирать к ним удачные заголовки.

**Задача-минимум**: создать сервис, который учит автора использовать разумные подходы для составления говорящего заголовка. То есть такого заголовока, по которому читатель понимает, какую пользу он получит от прочтения статьи. Мы понимаем, что автору недостаточно оценки, с помощью сервиса он хотел бы получить конкретику, научиться распознавать индикаторы, по которым читатель выбирает статью. Эту проблему можно сформулировать в виде задачи распознавания именованных сущностей (англ. [Named Entity Recognition](https://en.wikipedia.org/wiki/Named-entity_recognition), NER). Распознанные именованные сущности можно далее вмесе с токенизированным текстом использовать для выставления условного балла от 0 до 10, позволяющего автору быстро оценить и скорректировать результат.

**Задача-максимум** (пока не решаем): генерация вариантов более качественных заголовков по тексту публикации или сочетанию чернового заголовка и краткого содержания.


## Инструментарий
- Python 3
- Библиотеки [spaCy 3.0](https://spacy.io/) и [transformers 4.6](https://huggingface.co/transformers/). Мы выбрали `spacy` так как это стабильная библиотека, ориентированная на конечное использование в коммерческих приложениях.
- Label Studio для разметки эталонного набора. 

## План
- Составляем spaCy-пайплайн
- Выделяем 10 тыс. случайных заголовков, подлежащих разметке. При случайном выборе заголовки равномерно распределены по шкале баллов от 0 до 10
- Размечаем с помощью LabelStudio эталонный набор заголовков статей для NER. Исходно для стандартной русскоязычной модели уже имеются сущности `LOC`, `ORG`, `PER`.
- Обучаем пайплайн заголовков на NER-задаче
- Обучаем пайплайн на задаче регрессии

In [ ]:
# Чтобы проверить версию CUDA
#!nvcc --version
# В зависимости от версии обновить номер плагина для CUDA
#!pip install -U spacy[cuda110,transformers,lookups]
# Модель для русского языка если не использовать transformers
#!python -m spacy download ru_core_news_lg

In [2]:
!pip install -U spacy[cuda110,transformers,lookups]

     |████████████████████████████████| 12.8MB 288kB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 1.1MB 39.6MB/s 
     |████████████████████████████████| 460kB 41.2MB/s 
     |████████████████████████████████| 9.1MB 32.2MB/s 
     |████████████████████████████████| 165.3MB 81kB/s 
     |████████████████████████████████| 93.4MB 117kB/s 
     |████████████████████████████████| 92kB 12.4MB/s 
     |████████████████████████████████| 122kB 53.5MB/s 
     |████████████████████████████████| 2.1MB 28.3MB/s 
     |████████████████████████████████| 1.0MB 30.9MB/s 
     |████████████████████████████████| 3.3MB 31.1MB/s 
     |████████████████████████████████| 901kB 42.7MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=3fe20a74cafe23a3c83546d6499963cd1d6722a61bcbdf71c3dbe80896c35002
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successf

In [3]:
!python -m spacy download ru_core_news_lg

2021-05-23 08:54:28.746596: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 515.3MB 30kB/s 
     |████████████████████████████████| 61kB 5.6MB/s 
     |████████████████████████████████| 8.2MB 9.6MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [4]:
# исключение для проверки, на месте ли CUDA или мы учимся на CPU
from cupy.cuda.runtime import CUDARuntimeError
import spacy

try:
  spacy.prefer_gpu()
  print('CUDA GPU is used')
except CUDARuntimeError:
  spacy.require_cpu()
  print('CPU is used')

nlp = spacy.load("ru_core_news_lg")

CPU is used


In [ ]:
# # Construction via add_pipe with custom config
# config = {
#     "model": {
#         "@architectures": "spacy-transformers.TransformerModel.v1",
#         "name": "bert-base-multilingual-cased",
#         "tokenizer_config": {"use_fast": True}
#     }
# }

# trf = nlp.add_pipe("transformer",
#                    config=config)

In [5]:
nlp = ru_core_news_lg.load()
doc = nlp("Пример прекрасного текста.")
print([(w.text, w.pos_) for w in doc])

[('Пример', 'NOUN'), ('прекрасного', 'ADJ'), ('текста', 'NOUN'), ('.', 'PUNCT')]


In [6]:
doc = nlp("Москва, Кремль, пупырышка")

for ent in doc.ents:
  print(ent.text, ent.start_char, ent.end_char, ent.label_)

Москва 0 6 LOC
Кремль 8 14 LOC


Кроме имеющихся именованных сущностей мы добавляем следующие категории и подкатегории:
1. **Объект** `OBJ`. *О чём* эта статья.  Если заголовок состоит только из таких сущностей, значит перед нами что-то вроде статьи из словаря, объяснение сущности самого объекта. Самое то для тех, кто хочет разобраться что это и зачем нужно. Примеры: "LESS: программируемый язык стилей", "Composer — менеджер зависимостей для PHP".
2. **Аудитория** `AUD`. Для кого написан этот текст. Примеры индикаторов аудитории: "для новичков, на Windows, профи, любой аккаунт, до 30 лет, русская версия" Аудитория выражается и просто через "я" — мы сравниваем себя с другими людьми через наш общий или различный опыт. Наиболее читаемые статьи обращаются к аудитории новичков, но это не значит, что их читают только новички. "Пайка для начинающих", "Hello World-проект на Flask", "Основы IP-телефонии", "Какой язык программирования стоит выучить пер
вым?".
3. **Польза**. Какую проблему показывает или решает публикация. В чём ее профит?
Польза может выражаться самыми разными способами:
  + **Маркеры типа текста** `TYPE`: инструкция ("как установить", "Шаблон базовой настройки маршрутизатора Cisco"), определение ("что такое... и с чем едят"), новость (Новое в Java 8"), личный опыт, сравнение объектов ("X или Y", "Python vs R") и т. д. По маркеру типа текста мы понимаем, с чем имеем дело.
  + **Указание числа используемых источников или рассматриваемых объектов** `NUM`: "10 лучших", "ТОП-3".
  + **Усилия и время, которые потратит читатель на саму статью или процесс** `EFFORT`: "За 15 минут, за один вечер, за один год, краткое руководство, в 11 строчек кода". Вполне возможно, что у человека достаточно времени, и он хочет детально во всём разобраться: "Подробно о..., всё про...". Главное, что вся нужная информация нашлась в одном месте.
  + **Маркеры последовательного подхода, нового типа изложения** `STRUCT`. В интернете не хватает структурированной информации, люди любят когда рассказывают "по порядку, детально, без воды".
  + **Предостережение об опасности или возможной ошибке** `DANGER`: "Проблема в ... и ее их решение", "Взлом... от которого не спасёт", " "X – ловушка для неопытных. Осторожно".
  + **Маркировка акта длинного повествования** `PART`. Указание части в заголовке подсказывает: перед нами часть большого текста. Хорошо работает следующий формат: "Общее название группы технологий. Часть N. Название технологии."  Примеры: "jQuery для начинающих. Часть 3. AJAX". "Bash-скрипты, часть 2: циклы". "Пишем игры на C++, Часть 1/3 — Написание мини-фреймворка", "Сети для самых маленьких. Часть шестая. Динамическая маршрутизация".
4. **Источник движения** — в хороших статьях заложена история путешествия, они приводят читателя из пункта А в пункт Б. Саму историю расскажет статья, но полезно прочертить вектор с помощью глагола, или если придется к месту — искренней эмоции.
  + **Побуждение к действию или само действие** `TODO`. Что мы будем делать в этой статье. "Пишем программу...", "настройка, обзор, запуск, ремонт". Примеры: "Извлекаем золото из старой электроники", "Запуск старых игр на Windows".
  + **Эмоция** `EMO`. С эмоциями не стоит перебарщивать, но иногда сильная эмоция или выражение отношения — то, что нужно. "Xудшее, что могло с нами случиться." "Почему научиться программировать так чертовски тяжело?". Помните: читатель не дурак, эмоции в заголовке работают только, если они неподдельные.

Помните, что каким бы ни был заголовок, главное – сам текст и внимательное отношение к читателю.

In [7]:
nlp

In [8]:
nlp.vocab